In [ ]:
import torch, random, importlib, torchvision
import numpy as np

import dataset
import utils
import net

from PIL import ImageDraw, Image
from IPython.display import display
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm import tqdm

torch.set_printoptions(precision=3, linewidth=300)

In [ ]:
importlib.reload(utils)
importlib.reload(dataset)

#dataset_path = 'Datasets/Professor_Material/real_images'
dataset_path = 'Datasets/Generated_Dataset/val'

cv = utils.Converter()
d = dataset.RisikoDataset(dataset_dir=dataset_path, cv=cv, is_trainset=False)

In [ ]:
model = torch.load('weights/best_model_49').to('cpu')
model.eval()

In [ ]:
prob_thresh = 0.6
max_iou = 0.75

importlib.reload(utils)

def draw_boxes_on_image(img:Image, boxes:torch.Tensor):
    colors = ("blue", "red", "yellow", "purple", "black", "green")

    if (boxes.shape[0] != 0):

        if len(boxes.shape) == 1:
            boxes = boxes.unsqueeze(1)

        box_coords = boxes[:, 2:]

        img_draw = ImageDraw.Draw(img)

        for i in range(box_coords.shape[0]):

            x0 = int(box_coords[i,0] - box_coords[i,2] / 2)
            x1 = int(box_coords[i,0] + box_coords[i,2] / 2)
            y0 = int(box_coords[i,1] - box_coords[i,3] / 2)
            y1 = int(box_coords[i,1] + box_coords[i,3] / 2)

            img_draw.rectangle([x0, y0, x1, y1], outline=colors[int(boxes[i,1])%6])

mean_ap = MeanAveragePrecision(box_format='cxcywh', iou_type='bbox')
def get_single_score(pred_labels:torch.Tensor, target_labels:torch.Tensor, mean_ap_obj:MeanAveragePrecision, netin_shape:int) -> dict:

    preds, target = list(), list()
    preds_i, target_i = dict(), dict()

    # get labels absolute values
    target_i['boxes'] = target_labels[:,1:]
    target_i['labels'] = target_labels[:,0].type(torch.int32)

    preds_i['boxes'] = pred_labels[:,2:]
    preds_i['scores'] = pred_labels[:,0]
    preds_i['labels'] = pred_labels[:,1].type(torch.int32)

    target.append(target_i)
    preds.append(preds_i)

    return mean_ap_obj.forward([preds_i], [target_i])


rnd_idx = random.randint(0, len(d))
img_tensor, target = d.__getitem__(idx=rnd_idx)
target = target.clone()
cv.convert_labels_from_relative_to_absolute_values(target)

model.to('cpu')
netout = model.forward(img_tensor.unsqueeze(0)).squeeze(0)

print('Total number of detections above probability ' + str(prob_thresh) + ' is ' + str(torch.count_nonzero(netout > prob_thresh).item()))

preds_labels = cv.convert_netout_to_labels(netout=netout, probability_threshold=prob_thresh, max_iou=max_iou, apply_nms=True)
preds_labels_map = preds_labels.clone()

print('Number of detections after nms with maximum iou ' + str(max_iou) + ' is ' + str(preds_labels.shape[0]))
cv.convert_labels_from_relative_to_absolute_values(preds_labels[:,1:])
cv.convert_labels_from_relative_to_absolute_values(preds_labels_map[:,1:])
to_pil = torchvision.transforms.ToPILImage()
img = to_pil(d.invnorm(img_tensor).type(torch.uint8))

draw_boxes_on_image(img, preds_labels)

print(get_single_score(preds_labels_map, target, mean_ap, netin_shape=cv.netin_img_shape))

display(img)

In [ ]:
# model.to('cuda')
# model.eval()

def get_score(model:net.GridNet, dataset:dataset.RisikoDataset, mean_ap_obj:MeanAveragePrecision, device:str='cpu', prob_threshold:float=0.5, iou_threshold:float=0.7) -> float:

    mAP = 0
    map_50 = 0
    mAP_75 = 0
    detections = 0
    effective_objs = 0

    assert dataset.train_mode == False

    pbar = tqdm(len(dataset), desc='\t Evaluating mAP on validation set', unit='sample', leave=True)
    pbar.set_postfix(mAP=0, map_50=0, mAP_75=0, avg_detect=0, avg_objs=0)

    for i in range(len(dataset)):
        img_tensor, target_labels = dataset.__getitem__(i)
        img_tensor = img_tensor.unsqueeze(0)
        target_labels = target_labels.clone() # clone to prevent modifications on original
        effective_objs += target_labels.shape[0]

        preds, target = list(), list()
        preds_i, target_i = dict(), dict()

        netout = model.forward(img_tensor.to(device)).to('cpu')

        pred_labels = dataset.cv.convert_netout_to_basic_labels(netout.squeeze(0), probability_threshold=prob_threshold, max_iou=iou_threshold, apply_nms=True)
        dataset.cv.convert_labels_from_relative_to_absolute_values(pred_labels[...,1:])
        detections += pred_labels.shape[0]

        # get labels absolute values
        target_i['boxes'] = target_labels[:,1:]
        target_i['labels'] = target_labels[:,0].type(torch.int32)

        preds_i['boxes'] = pred_labels[:,2:]
        preds_i['scores'] = pred_labels[:,0]
        preds_i['labels'] = pred_labels[:,1].type(torch.int32)


        mAP_dict = mean_ap_obj.forward([preds_i], [target_i])
        if mAP_dict['map'] != -1:
            mAP += mAP_dict['map']
        if mAP_dict['map_50'] != -1:
            map_50 += mAP_dict['map_50']
        if mAP_dict['map_75'] != -1:
            mAP_75 += mAP_dict['map_75']

        pbar.set_postfix(mAP=mAP.item()/(i+1), map_50=map_50.item()/(i+1), mAP_75=mAP_75.item()/(i+1), avg_detect=detections/(i+1), avg_objs=effective_objs/(i+1))
        pbar.update(1)

    return mAP.item()/len(dataset)

#get_score(model=model, dataset=d, mean_ap_obj=mean_ap, device='cpu', prob_threshold=0.5, iou_threshold=0.5)
